# Ross Sea synthetic gravity inversion

Here we try to recover the true bathymetry of the Ross Sea from a synthetic airborne gravity survey. See notebook `Ross_Sea_synthetic_model.ipynb` for the creation of this synthetic gravity data and the low-resolution starting model of bathymetry. 

import packages

In [1]:
%load_ext autoreload
%autoreload 2
%load_ext snakeviz

import sys
import logging
import copy
import zarr
import harmonica as hm
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import verde as vd
import scipy as sp
import pvxarray
import geopandas as gpd
import rioxarray
import pygmt
import plotly.graph_objects as go
import optuna
from optuna.storages import JournalStorage, JournalFileStorage
import warnings
import os
import pathlib
import string
import itertools
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import pickle
import string
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from tqdm.contrib.itertools import product
from tqdm.autonotebook import tqdm

sns.set_theme()

import sys

# local python scripts
import RIS_gravity_inversion.synthetic as RIS_synth
import RIS_gravity_inversion.plotting as RIS_plotting

from invert4geom import plotting, inversion, regional, synthetic, utils, optimization, cross_validation, uncertainty

from polartoolkit import regions, maps, fetch, profiles
from polartoolkit import utils as polar_utils
import zarr

import numpy as np
import pandas as pd
import xarray as xr
import verde as vd
import scipy as sp
import rioxarray
import pygmt
import copy
import harmonica as hm
import geopandas as gpd
from shapely.geometry import Polygon


import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings

import os

os.environ['POLARTOOLKIT_HEMISPHERE']='north'

# Load data

## topographic layers
* full resolution bed 
* low resolution bed

In [ ]:
bed = xr.open_zarr("../../synthetic_data/Ross_Sea_bed.zarr").z

starting_bed = xr.open_zarr(
    "../../synthetic_data/Ross_Sea_lowres_bed_semiregular.zarr"
).z

# ensure all xarray are same data type
bed = bed.astype(np.float64)
starting_bed = starting_bed.astype(np.float64)

print(polar_utils.get_grid_info(bed))
print(polar_utils.get_grid_info(starting_bed))
starting_bed

## gravity data
* airborne survey gravity data
* full resolution forward gravity of the regional field (basement)

In [ ]:
# true regional gravity anomaly
true_regional_grav = xr.open_zarr(
    "../../synthetic_data/Ross_Sea_forward_grav.zarr"
).basement_grav

# airborne survey gravity point data
grav_df = pd.read_csv(
    "../../synthetic_data/Ross_Sea_airborne_survey_gravity.csv.gz",
    sep=",",
    header="infer",
    index_col=None,
    compression="gzip",
)
grav_df

In [ ]:
grav_df.set_index(["northing", "easting"]).to_xarray().pred.plot()

# Set region and parameters

Use the same regions and parameters as `Ross_Sea synthetic_model.ipynb`

In [ ]:
layer_spacing, buffer_region, _, _, registration = polar_utils.get_grid_info(bed)

# set gravity observation parameters
grav_spacing = layer_spacing

inversion_region = vd.pad_region(buffer_region, pad=-40e3)

# set density contrast
density_contrast = 2300 - 1024

print(f"layer spacing: {layer_spacing}m")
print(f"registration: {registration}")
print("inversion region = ", inversion_region)
print("buffer region = ", buffer_region)

# Prep grav data
## grav data

In [ ]:
grav_df = grav_df.rename(
    columns={
        "pred": "Gobs",
    },
    )

# remove points outside of region of interest
grav_df = polar_utils.points_inside_region(
    grav_df, region=inversion_region, names=["easting", "northing"]
)

print(f"Gravity data: {len(grav_df)} points")
print(f"gravity avg. elevation: {int(np.nanmean(grav_df.upward))}")

## constraint points

In [ ]:
# load constraint points into a dataframe
constraints_all = pd.read_csv(
    "../../synthetic_data/Ross_Sea_synthetic_constraints_semiregular.csv.gz",
    sep=",",
    header="infer",
    index_col=None,
    compression="gzip",
)

# ensure all points are within inversion region
constraints = polar_utils.points_inside_region(
    constraints_all, inversion_region, names=("easting", "northing")
)

print(f"bathymetry control points:{len(constraints)}")
constraints.head()

## Gravity misfit

The `gravity misfit` is defined as the difference between the observed and predicted gravity:

$G_{misfit} = G_{obs} - G_{forward}$ 

To determine $G_{misfit}$, we need to forward model the gravitational effect of the low-resolution bathymetry density contrast. This will be the input into our inversion.


## Bed prisms

In [ ]:
# create prisms around low-res bathymetry
# positive densities above reference, negative below
print(f"Bathymetry density contrast: {density_contrast}kgm-3")

zref = starting_bed.values.mean()

density_grid = xr.where(starting_bed >= zref, density_contrast, -density_contrast)

bed_prisms = utils.grids_to_prisms(
    surface=starting_bed,
    reference=zref,
    density=density_grid,
)

plotting.show_prism_layers(
    [bed_prisms],
    cmap="viridis",
    color_by="density",
    # color_by="thickness",
    zscale=100,
    clip_box=False,
    log_scale=False,
)

## Bed gravity

In [ ]:
# calculate gravity of starting bed
grav_df["starting_grav"] = bed_prisms.prism_layer.gravity(
    coordinates=(
        grav_df.easting,
        grav_df.northing,
        grav_df.upward,
    ),
    field="g_z",
    progressbar=True,
)

grav_df

# Misfit

In [ ]:
# calculate misfit as observed - starting
grav_df["misfit"] = grav_df.Gobs - grav_df.starting_grav
grav_df

### Save and load results

In [ ]:
grav_df.to_csv(
    "../../synthetic_data/Ross_Sea_starting_model_forward_gravity_survey.csv.gz",
    sep=",",
    na_rep="",
    header=True,
    index=False,
    encoding="utf-8",
    compression="gzip",
)

In [ ]:
grav_df = pd.read_csv(
    "../../synthetic_data/Ross_Sea_starting_model_forward_gravity_survey.csv.gz",
    sep=",",
    header="infer",
    index_col=None,
    compression="gzip",
)
grav_df

# Regional - Residual seperation

Here we seperate the gravity misfit into `regional` and `residual` components. Since we created the synthetic model, we precisely know the regional component (the forward gravity of the `basement` layer), and can thus tune our regional-residual seperation method to accurately remove the regional field. 

In [ ]:
block_kfold_split_df = cross_validation.split_test_train(
    constraints,
    method="KFold",
    # spacing=spacing,
    shape=(20,20),
    plot=True,
)
block_kfold_split_df

In [ ]:
# values needed in all the regional separation techniques
kwargs = {
    "grav_data_column":"misfit",
    "regional_column":"reg",
    # "score_as_median":True,
    "true_regional": true_regional_grav,
}

In [ ]:
study, grav_df = optimization.optimize_regional_constraint_point_minimization_kfolds(
    block_kfold_split_df,
    grav_df=grav_df,
    grid_method="eq_sources",
    source_depth_limits=[10e3, 200e3],
    block_size_limits=[1e3, 50e3],
    eq_damping_limits=[0, 10],
    grav_obs_height_limits=[0, 50e3],

    n_trials=100,
    plot=True,
    optimize_on_true_regional_misfit=True,
    **kwargs,
)


In [ ]:
_ = polar_utils.grd_compare(
    true_regional,
    grav_df.set_index(["northing", "easting"]).to_xarray().reg,
    grid1_name="True regional",
    grid2_name="Calculated regional",
    plot=True,
    region=inversion_region,
    points=constraints.rename(columns={"easting": "x", "northing": "y"}),
    title="difference",
    plot_type="pygmt",
    hist=True,
    inset=False,
    points_style="x.15c",
    subplot_labels=True,
    robust=False,
    verbose="q",
    cbar_label="mGal",
)

In [ ]:
study, grav_df = optimization.optimize_regional_constraint_point_minimization_kfolds(
    block_kfold_split_df,
    grav_df=grav_df,
    grid_method="eq_sources",
    source_depth_limits=[10e3, 200e3],
    eq_damping_limits=[0, 10],
    # if limits are omitted, must provide constant values
    block_size=10e3,
    grav_obs_height=20e3,

    n_trials=100,
    plot=True,
    # optimize_on_true_regional_misfit=True,
    **kwargs,
)


In [ ]:
_ = polar_utils.grd_compare(
    true_regional,
    grav_df.set_index(["northing", "easting"]).to_xarray().reg,
    grid1_name="True regional",
    grid2_name="Calculated regional",
    plot=True,
    region=inversion_region,
    points=constraints.rename(columns={"easting": "x", "northing": "y"}),
    title="difference",
    plot_type="pygmt",
    hist=True,
    inset=False,
    points_style="x.15c",
    subplot_labels=True,
    robust=False,
    verbose="q",
    cbar_label="mGal",
)

In [ ]:
_ = RIS_plotting.anomalies_plotting(
    grav_df,
    title=f" Constraint Point Minimization Regional Separation",
    constraints=constraints.rename(columns={"easting":"x","northing":"y"}),
    input_forward_column="starting_grav",
    input_grav_column="Gobs",
    plot_type="pygmt",
    robust=False,
)


### Save and load results

In [ ]:
grav_df.to_csv(
    "../../synthetic_data/Ross_Sea_gravity_anomalies_survey.csv.gz",
    sep=",",
    na_rep="",
    header=True,
    index=False,
    encoding="utf-8",
    compression="gzip",
)

In [ ]:
grav_df = pd.read_csv(
    "../../synthetic_data/Ross_Sea_gravity_anomalies_survey.csv.gz",
    sep=",",
    header="infer",
    index_col=None,
    compression="gzip",
)
grav_df

# Constraints grid

In [ ]:
starting_prisms = copy.deepcopy(bed_prisms)

min_dist = utils.normalized_mindist(
    constraints,
    starting_prisms,
    mindist=layer_spacing / np.sqrt(2),
    low=0,
    high=1,
    region=inversion_region,
)
starting_prisms["weights"] = min_dist
starting_prisms.weights.plot()

In [ ]:
enc = {x: {"compressor": zarr.Blosc()} for x in starting_prisms}
starting_prisms.to_zarr(
    "../../inversion_layers/Ross_Sea_starting_prisms.zarr",
    encoding=enc,
    mode="w",
)

In [ ]:
starting_prisms = xr.open_zarr(
    "../../inversion_layers/Ross_Sea_starting_prisms.zarr",
)

# Inversion

Now that we have $G_{misfit}$, we can run an inversion to attempt to recover the true bathymetry. 

In [ ]:
# set Python's logging level
logger = logging.getLogger()
logger.setLevel(logging.WARNING)

# set kwargs to pass to the inversion
kwargs = {
    "grav_data_column": "Gobs",
    "prism_layer": starting_prisms,
    "deriv_type": "annulus",
    # set stopping criteria
    "max_iterations": 100,
    "l2_norm_tolerance": 0.01,
    "delta_l2_norm_tolerance": 1.02,
}

# set which damping parameters to include
dampings = np.logspace(-4, 0, 10)

best_inv_results, best_damping, _, _, scores = cross_validation.grav_optimal_parameter(
    training_data=grav_df[grav_df.test == False],  # noqa: E712
    testing_data=grav_df[grav_df.test == True],  # noqa: E712
    param_to_test=("solver_damping", dampings),
    progressbar=True,
    # plot_grids=True,
    plot_cv=False,
    verbose=True,
    **kwargs,
)

In [ ]:
# Compare the scores and the damping values
plotting.plot_cv_scores(
    scores,
    dampings,
    param_name="Damping",
    logx=True,
    logy=True,
)

In [ ]:
# collect the results
topo_results, grav_results, parameters, elapsed_time = best_inv_results

plotting.plot_convergence(
    grav_results,
    iter_times=parameters["Iteration times"],
)

plotting.plot_inversion_results(
    grav_results,
    topo_results,
    parameters,
    inversion_region,
    iters_to_plot=2,
    plot_iter_results=True,
    plot_topo_results=True,
    plot_grav_results=True,
    constraints_df=constraints,
)

final_topography = topo_results.set_index(["northing", "easting"]).to_xarray().topo

_ = polar_utils.grd_compare(
    bed,
    final_topography,
    region=inversion_region,
    plot=True,
    grid1_name="True topography",
    grid2_name="Inverted topography",
    robust=True,
    hist=True,
    inset=False,
    verbose="q",
    title="difference",
    grounding_line=False,
    reverse_cpt=True,
    cmap="rain",
    points=constraints.rename(columns={"easting": "x", "northing": "y"}),
    points_style="x.15c",
)

In [ ]:
RIS_plotting.plot_inversion_results_profile(
    topo_results,
    grav_results,
    observed_grav_column="Gobs",
    constraints=constraints[constraints.inside],
    true_surface=bed,
    start=[inversion_region[0], inversion_region[3]],
    stop=[inversion_region[1], inversion_region[2]],
    map_buffer=0.02,
    subplot_orientation="horizontal",
    data_legend_loc="jBL+o0c",
    layers_legend_loc="jBR+o0c",
    layers_frame=["nEsw", "xafg", "ya+lElevation (m)"],
    data_frame=["nEsw", "xafg", "ya+lGravity (mGal)"],
)

In [ ]:
# inverted bed error
final_topography = final_topography.sel(
    easting=slice(inversion_region[0], inversion_region[1]),
    northing=slice(inversion_region[2], inversion_region[3]),
)
true_bed = bed.sel(
    easting=slice(inversion_region[0], inversion_region[1]),
    northing=slice(inversion_region[2], inversion_region[3]),
)
bed_dif = true_bed - final_topography

# regional separation error
ds = (
    grav_results[grav_results.test == False]
    .set_index(["northing", "easting"])
    .to_xarray()
)
true_reg = true_regional_grav
reg_dif = true_reg - ds.reg
reg_dif -= np.nanmean(reg_dif)

fig = maps.plot_grd(
    bed_dif,
    fig_height=10,
    title="Inverted bed error",
    hist=True,
    points=constraints[constraints.inside].rename(
        columns={"easting": "x", "northing": "y"}
    ),
    points_style="x.2c",
    points_pen="1.5p",
    cmap="balance+h0",
    cbar_label="m",
    cbar_yoffset=1.5,
)
fig.text(
    position="TL",
    justify="BL",
    text="a)",
    font="16p,Helvetica,black",
    offset="j-.3/.3",
    no_clip=True,
)
fig.plot(
    x=constraints[constraints.inside == False].easting,
    y=constraints[constraints.inside == False].northing,
    style="x.05c",
)

fig = maps.plot_grd(
    reg_dif,
    title="Regional gravity error",
    cmap=f"balance+h0",
    reverse_cpt=True,
    fig=fig,
    hist=True,
    points=constraints[constraints.inside].rename(
        columns={"easting": "x", "northing": "y"}
    ),
    points_style="x.2c",
    points_pen="1.5p",
    origin_shift="xshift",
    cbar_label="mGal",
    cbar_yoffset=1.5,
)

fig.text(
    position="TL",
    justify="BL",
    text="b)",
    font="16p,Helvetica,black",
    offset="j-.3/.3",
    no_clip=True,
)
fig.plot(
    x=constraints[constraints.inside == False].easting,
    y=constraints[constraints.inside == False].northing,
    style="x.05c",
)
fig.show()

# Repeat with noisey survey data

## Regional separation

In [ ]:
# contaminate with noise
grav_df_noise = grav_df.copy()
grav_df_noise["Gobs"], _ = synthetic.contaminate(
    grav_df_noise.Gobs,
    stddev=0.02,
    percent=True,
    seed=0,
)
# calculate misfit as observed - starting
grav_df_noise["misfit"] = grav_df_noise.Gobs - grav_df_noise.starting_grav

kwargs = {
    "grav_data_column":"misfit",
    "regional_column":"reg",
    # "score_as_median":True,
    "true_regional": true_regional_grav,
}

study, grav_df_noise = optimization.optimize_regional_constraint_point_minimization_kfolds(
    block_kfold_split_df,
    grav_df=grav_df_noise,
    grid_method="eq_sources",
    source_depth_limits=[10e3, 200e3],
    eq_damping_limits=[0, 10],
    # if limits are omitted, must provide constant values
    block_size=10e3,
    grav_obs_height=20e3,
    n_trials=100,
    plot=True,
    # optimize_on_true_regional_misfit=True,
    **kwargs,
)

_ = polar_utils.grd_compare(
    true_regional_grav,
    grav_df_noise.set_index(["northing", "easting"]).to_xarray().reg,
    grid1_name="True regional",
    grid2_name="Calculated regional",
    plot=True,
    region=inversion_region,
    points=constraints.rename(columns={"easting": "x", "northing": "y"}),
    title="difference",
    plot_type="pygmt",
    hist=True,
    inset=False,
    points_style="x.15c",
    subplot_labels=True,
    robust=False,
    verbose="q",
    cbar_label="mGal",
)

_ = RIS_plotting.anomalies_plotting(
    grav_df_noise,
    title=f" Constraint Point Minimization Regional Separation",
    constraints=constraints.rename(columns={"easting":"x","northing":"y"}),
    input_forward_column="starting_grav",
    input_grav_column="Gobs",
    plot_type="pygmt",
    robust=False,
)


In [ ]:
ds = (
    grav_df_noise[grav_df_noise.test == False]
    .set_index(["northing", "easting"])
    .to_xarray()
)

fig = maps.plot_grd(
    true_regional_grav,
    fig_height=10,
    cmap="balance+h0",
    title="True regional",
    cbar_label="mGal",
    frame=["nSWe", "xaf10000", "yaf10000"],
    hist=True,
    cbar_yoffset=2,
    points=constraints[constraints.inside].rename(
        columns={"easting": "x", "northing": "y"}
    ),
    points_style="x.2c",
    points_pen="1.5p",
)
fig.text(
    position="TL",
    justify="BL",
    text="a)",
    font="16p,Helvetica,black",
    offset="j-.3/.3",
    no_clip=True,
)
fig.plot(
    x=constraints[constraints.inside == False].easting,
    y=constraints[constraints.inside == False].northing,
    style="x.05c",
)

dif = true_regional_grav - ds.reg

fig = maps.plot_grd(
    dif,
    fig_height=10,
    cmap="balance+h0",
    title=f"RMSE {round(utils.rmse(dif),1)} mGal",
    cbar_label="mGal",
    hist=True,
    cbar_yoffset=2,
    grd2_cpt=True,
    fig=fig,
    origin_shift="xshift",
    xshift_amount=1.1,
    points=constraints[constraints.inside].rename(
        columns={"easting": "x", "northing": "y"}
    ),
    points_style="x.2c",
    points_pen="1.5p",
)
fig.text(
    position="TL",
    justify="BL",
    text="b)",
    font="16p,Helvetica,black",
    offset="j-.3/.3",
    no_clip=True,
)
fig.plot(
    x=constraints[constraints.inside == False].easting,
    y=constraints[constraints.inside == False].northing,
    style="x.05c",
)

fig = maps.plot_grd(
    ds.reg,
    fig_height=10,
    cmap="balance+h0",
    title="Calculated regional",
    cbar_label="mGal",
    frame=["nSwe", "xaf10000", "yaf10000"],
    hist=True,
    cbar_yoffset=2,
    fig=fig,
    origin_shift="xshift",
    xshift_amount=1.1,
    points=constraints[constraints.inside].rename(
        columns={"easting": "x", "northing": "y"}
    ),
    points_style="x.2c",
    points_pen="1.5p",
)
fig.text(
    position="TL",
    justify="BL",
    text="c)",
    font="16p,Helvetica,black",
    offset="j-.3/.3",
    no_clip=True,
)
fig.plot(
    x=constraints[constraints.inside == False].easting,
    y=constraints[constraints.inside == False].northing,
    style="x.05c",
)

fig.show()

In [ ]:
grav_df_noise.to_csv(
    "../../synthetic_data/Ross_Sea_gravity_anomalies_survey_noise.csv.gz",
    sep=",",
    na_rep="",
    header=True,
    index=False,
    encoding="utf-8",
    compression="gzip",
)

In [ ]:
grav_df_noise = pd.read_csv(
    "../../synthetic_data/Ross_Sea_gravity_anomalies_survey_noise.csv.gz",
    sep=",",
    header="infer",
    index_col=None,
    compression="gzip",
)
grav_df_noise

### Plot misfit grids

In [ ]:
grav_grids = (
    grav_df_noise[grav_df_noise.test == False]
    .set_index(["northing", "easting"])
    .to_xarray()
)

fig = maps.plot_grd(
    grav_grids.misfit,
    fig_height=10,
    cmap="balance+h0",
    title="Gravity misfit",
    cbar_label="mGal",
    frame=["nSWe", "xaf10000", "yaf10000"],
    hist=True,
    cbar_yoffset=2,
    points=constraints[constraints.inside].rename(
        columns={"easting": "x", "northing": "y"}
    ),
    points_style="x.2c",
    points_pen="1.5p",
)
fig.text(
    position="TL",
    justify="BL",
    text="a)",
    font="16p,Helvetica,black",
    offset="j-.3/.3",
    no_clip=True,
)
fig.plot(
    x=constraints[constraints.inside == False].easting,
    y=constraints[constraints.inside == False].northing,
    style="x.05c",
)

fig = maps.plot_grd(
    grav_grids.reg,
    fig_height=10,
    cmap="balance+h0",
    title=f"Regional",
    cbar_label="mGal",
    hist=True,
    cbar_yoffset=2,
    grd2_cpt=True,
    fig=fig,
    origin_shift="xshift",
    xshift_amount=1.1,
    points=constraints[constraints.inside].rename(
        columns={"easting": "x", "northing": "y"}
    ),
    points_style="x.2c",
    points_pen="1.5p",
)
fig.plot(
    x=constraints[constraints.inside == False].easting,
    y=constraints[constraints.inside == False].northing,
    style="x.05c",
)
# plot profiles location, and endpoints on map
start = [inversion_region[0], inversion_region[3]]
stop = [inversion_region[1], inversion_region[2]]
fig.plot(
    vd.line_coordinates(start, stop, size=100),
    pen="2p,black",
)
fig.text(
    x=start[0],
    y=start[1],
    text="A",
    fill="white",
    font="12p,Helvetica,black",
    justify="CM",
    clearance="+tO",
    no_clip=True,
)
fig.text(
    x=stop[0],
    y=stop[1],
    text="A' ",
    fill="white",
    font="12p,Helvetica,black",
    justify="CM",
    clearance="+tO",
    no_clip=True,
)
fig.text(
    position="TL",
    justify="BL",
    text="b)",
    font="16p,Helvetica,black",
    offset="j-.3/.3",
    no_clip=True,
)


# plot the synthetic observed gravity grid
fig = maps.plot_grd(
    grav_grids.res,
    fig_height=10,
    cmap="balance+h0",
    title="Residual",
    cbar_label="mGal",
    frame=["nSwe", "xaf10000", "yaf10000"],
    hist=True,
    cbar_yoffset=2,
    fig=fig,
    origin_shift="xshift",
    xshift_amount=1.1,
    points=constraints[constraints.inside].rename(
        columns={"easting": "x", "northing": "y"}
    ),
    points_style="x.2c",
    points_pen="1.5p",
)
fig.text(
    position="TL",
    justify="BL",
    text="c)",
    font="16p,Helvetica,black",
    offset="j-.3/.3",
    no_clip=True,
)
fig.plot(
    x=constraints[constraints.inside == False].easting,
    y=constraints[constraints.inside == False].northing,
    style="x.05c",
)

fig.show()

### plot profiles

In [ ]:
grav_grids = (
    grav_df_noise[grav_df_noise.test == False]
    .set_index(["northing", "easting"])
    .to_xarray()
)

data_grids = [
    grav_grids.misfit,
    grav_grids.reg,
    grav_grids.res,
]

data_names = [
    "Gravity misfit",
    "Regional",
    "Residual",
]

data_dict = profiles.make_data_dict(
    names=data_names,
    grids=data_grids,
    colors=[
        # "cyan",
        "red",
        "black",
        "blue",
        # "purple",
    ],
    axes=[0, 0, 0],
)
fig, df_data = profiles.plot_data(
    "points",
    start=[inversion_region[0], inversion_region[3]],
    stop=[inversion_region[1], inversion_region[2]],
    num=10000,
    fig_height=4,
    fig_width=15,
    data_dict=data_dict,
    data_legend_loc="jBL+jBL",  # +o0c/.4c",
    data_buffer=0.1,
    data_frame=["neSW", "yag+lgravity (mGal)", "xag+lDistance (m)"],
    data_legend_box="+gwhite+c1p+p1p",
    share_yaxis=True,
    start_label="A",
    end_label="A' ",
)
fig.show()

In [ ]:
# get grid of min dists to nearest constraints
min_dist = utils.normalized_mindist(
    constraints,
    starting_prisms,
)
df_data = profiles.sample_grids(df_data, min_dist, interpolation="b", sampled_name="min_dist")

# df_data.describe()

In [ ]:
data_max = np.max(df_data[["0", "1", "2"]].values) * 1.5

# series = utils.get_min_max(df_data.min_dist, robust=False)
series = [1e3, 10e3]
pygmt.makecpt(
    cmap="gray",
    series=series,
    background=True,
)
fig.plot(
    x=df_data.dist,
    y=np.ones_like(df_data.dist) * data_max,
    style="c0.1c",
    fill=df_data.min_dist,
    cmap=True,
    no_clip=True,
)
fig.colorbar(
    cmap=True,
    position="jMR+w2c/.3c+jML+v+o0.2c/0c",
    frame=[f"xa+lnearest constraint", "y+lkm"],
    scale=0.001,
)

fig.show()

## inversion

In [ ]:
# set Python's logging level
logger = logging.getLogger()
logger.setLevel(logging.WARNING)

# set kwargs to pass to the inversion
kwargs = {
    "grav_data_column": "Gobs",
    "prism_layer": starting_prisms,
    "deriv_type": "annulus",
    # set stopping criteria
    "max_iterations": 100,
    "l2_norm_tolerance": 0.01,
    "delta_l2_norm_tolerance": 1.02,
}

# set which damping parameters to include
dampings = np.logspace(-4, 0, 10)

best_inv_results, best_damping, _, _, scores = cross_validation.grav_optimal_parameter(
    training_data=grav_df_noise[grav_df_noise.test == False],  # noqa: E712
    testing_data=grav_df_noise[grav_df_noise.test == True],  # noqa: E712
    param_to_test=("solver_damping", dampings),
    progressbar=True,
    # plot_grids=True,
    plot_cv=False,
    verbose=True,
    **kwargs,
)

In [ ]:
# Compare the scores and the damping values
plotting.plot_cv_scores(
    scores,
    dampings,
    param_name="Damping",
    logx=True,
    logy=True,
)

In [ ]:
# collect the results
topo_results, grav_results, parameters, elapsed_time = best_inv_results

plotting.plot_convergence(
    grav_results,
    iter_times=parameters["Iteration times"],
)

plotting.plot_inversion_results(
    grav_results,
    topo_results,
    parameters,
    inversion_region,
    iters_to_plot=2,
    plot_iter_results=True,
    plot_topo_results=True,
    plot_grav_results=True,
    constraints_df=constraints,
)

final_topography = topo_results.set_index(["northing", "easting"]).to_xarray().topo

_ = polar_utils.grd_compare(
    bed,
    final_topography,
    region=inversion_region,
    plot=True,
    grid1_name="True topography",
    grid2_name="Inverted topography",
    robust=True,
    hist=True,
    inset=False,
    verbose="q",
    title="difference",
    grounding_line=False,
    reverse_cpt=True,
    cmap="rain",
    points=constraints.rename(columns={"easting": "x", "northing": "y"}),
    points_style="x.15c",
)

In [ ]:
# inverted bed error
final_topography = final_topography.sel(
    easting=slice(inversion_region[0], inversion_region[1]),
    northing=slice(inversion_region[2], inversion_region[3]),
)
true_bed = bed.sel(
    easting=slice(inversion_region[0], inversion_region[1]),
    northing=slice(inversion_region[2], inversion_region[3]),
)
bed_dif = true_bed - final_topography

# regional separation error
ds = (
    grav_results[grav_results.test == False]
    .set_index(["northing", "easting"])
    .to_xarray()
)
true_reg = true_regional_grav
reg_dif = true_reg - ds.reg
reg_dif -= np.nanmean(reg_dif)

fig = maps.plot_grd(
    bed_dif,
    fig_height=10,
    title="Inverted bed error",
    hist=True,
    points=constraints[constraints.inside].rename(
        columns={"easting": "x", "northing": "y"}
    ),
    points_style="x.2c",
    points_pen="1.5p",
    cmap="balance+h0",
    cbar_label=f"m (RMSE={int(utils.rmse(bed_dif))})",
    cbar_yoffset=1.5,
)
fig.text(
    position="TL",
    justify="BL",
    text="a)",
    font="16p,Helvetica,black",
    offset="j-.3/.3",
    no_clip=True,
)
fig.plot(
    x=constraints[constraints.inside == False].easting,
    y=constraints[constraints.inside == False].northing,
    style="x.05c",
)

fig = maps.plot_grd(
    reg_dif,
    title="Regional gravity error",
    cmap=f"balance+h0",
    reverse_cpt=True,
    fig=fig,
    hist=True,
    points=constraints[constraints.inside].rename(
        columns={"easting": "x", "northing": "y"}
    ),
    points_style="x.2c",
    points_pen="1.5p",
    origin_shift="xshift",
    cbar_label=f"mGal (RMSE={int(utils.rmse(reg_dif))})",
    cbar_yoffset=1.5,
)

fig.text(
    position="TL",
    justify="BL",
    text="b)",
    font="16p,Helvetica,black",
    offset="j-.3/.3",
    no_clip=True,
)
fig.plot(
    x=constraints[constraints.inside == False].easting,
    y=constraints[constraints.inside == False].northing,
    style="x.05c",
)
fig.show()

### plot profiles

In [ ]:
prism_ds = topo_results.set_index(["northing", "easting"]).to_xarray()
grav_ds = grav_results.set_index(["northing", "easting"]).to_xarray()

initial_topo = prism_ds.starting_topo

final_topo = prism_ds.topo

cols = [s for s in grav_results.columns.to_list() if "_initial_misfit" in s]
initial_residual = grav_ds[cols[0]]

cols = [s for s in grav_results.columns.to_list() if "_forward_grav" in s]
final_forward = grav_ds[cols[-1]]

observed = grav_ds.Gobs

topo_names = [
    "Starting bed",
    "Inverted bed",
    "True bed",
]
topo_grids = [
    initial_topo,
    final_topo,
    bed,
]
layers_dict = profiles.make_data_dict(
    names=topo_names,
    grids=topo_grids,
    colors=["black", "blue", "red"],
)

data_names = [
    "Initial residual gravity",
    "Final forward gravity",
    "Observed gravity",
]

data_grids = [
    initial_residual,
    final_forward + grav_ds.reg,
    observed,
]

data_dict = profiles.make_data_dict(
    names=data_names,
    grids=data_grids,
    colors=[
        "black",
        "blue",
        "red",
    ],
)

In [ ]:
fig, df_layers, df_data = profiles.plot_profile(
    "points",
    start=[inversion_region[0], inversion_region[3]],
    stop=[inversion_region[1], inversion_region[2]],
    num=10000,
    fig_height=8,
    data_height=4,
    fig_width=14,
    data_dict=data_dict,
    data_legend_loc="jBL+jBL",  # +o0c/.4c",
    data_legend_box="+gwhite+c0c+p1p",
    data_buffer=0.1,
    data_frame=["neSW", "xafg+lDistance (m)", "yag+lgravity (mGal)"],
    data_pen_style=[None, None, "4_2:2p"],
    data_pen_thickness=[1, 1.5, 1],
    share_yaxis=True,
    layers_dict=layers_dict,
    layers_legend_loc="jBC+jBC",  # +o0c/-.2c",
    layers_legend_box="+gwhite+c0c+p1p",
    fill_layers=False,
    layers_frame=["nesW", "xafg", "yag+lelevation (m)"],
    layer_pen_thickness=[1, 1.5, 1],
    layers_pen_style=[None, None, "4_2:2p"],
    start_label="A",
    end_label="A' ",
)
fig.show()

In [ ]:
# get grid of min dists to nearest constraints
min_dist = utils.normalized_mindist(
    constraints,
    starting_prisms,
)
df_layers = profiles.sample_grids(
    df_layers, min_dist, interpolation="b", sampled_name="min_dist"
)

In [ ]:
data_max = np.max(df_layers[["0", "1", "2"]].values) * 0.6

# series = utils.get_min_max(df_data.min_dist, robust=False)
series = [1e3, 10e3]
pygmt.makecpt(
    cmap="gray",
    series=series,
    background=True,
)
fig.plot(
    x=df_layers.dist,
    y=np.ones_like(df_layers.dist) * data_max,
    style="c0.1c",
    fill=df_layers.min_dist,
    cmap=True,
    no_clip=True,
)
fig.colorbar(
    cmap=True,
    position="jMR+w2c/.3c+jML+v+o0.2c/0c",
    frame=[f"xa+lnearest constraint", "y+lkm"],
    scale=0.001,
)

fig.show()

In [ ]:
dif = bed - final_topo
dif_masked = polar_utils.mask_from_shp(
    "../../data/Ross_Sea_outline.shp",
    xr_grid=dif,
    masked=True,
    invert=False,
)

In [ ]:
titles = [
    "True bathymetry",
    f"Difference, RMSE: {int(utils.rmse(dif_masked))} m",
    "Inverted bathymetry",
]
cbar_labels = [
    "elevation (m)",
    "difference (m)",
    "elevation (m)",
]
grids = [
    bed,
    dif_masked,
    final_topo,
]

lims = [-800, -200]
pygmt.grd2cpt(
    grid=final_topo,
    cmap="batlowW",
    limit=lims,
    nlevels=13,
    output="../../plotting/tmp.cpt",
)

region = inversion_region

for i, g in enumerate(grids):
    if i == 0:
        fig = None
        origin_shift = "initialize"
        xshift_amount = 1
    else:
        fig = fig
        origin_shift = "xshift"
        xshift_amount = 1

    if i == 1:
        cmap = "balance+h0"
        # lims = utils.get_min_max(
        #     g,
        #     robust=True,
        # )
        # lims = (-vd.maxabs(lims), vd.maxabs(lims))
        lims = [-100, 100]
    else:
        cmap = "../../plotting/tmp.cpt"
        lims = [-800, -200]

    fig = maps.plot_grd(
        grid=g,
        fig_height=10,
        title=titles[i],
        title_font="16p,Helvetica,black",
        region=region,
        cmap=cmap,
        cpt_lims=lims,
        cbar_label=cbar_labels[i],
        # frame=frame,
        hist=True,
        hist_bin_num=50,
        cbar_yoffset=1,
        fig=fig,
        origin_shift=origin_shift,
        xshift_amount=xshift_amount,
    )
    if i == 1:
        fig.grdcontour(
            grid=g,
            interval=25,
            annotation=None,
            transparency=20,
            # resample=10e3,
            # cut="1C",
        )
        # plot profiles location, and endpoints on map
        start = [inversion_region[0], inversion_region[3]]
        stop = [inversion_region[1], inversion_region[2]]
        fig.plot(
            vd.line_coordinates(start, stop, size=100),
            pen="2p,black",
        )
        fig.text(
            x=start[0],
            y=start[1],
            text="A",
            fill="white",
            font="12p,Helvetica,black",
            justify="CM",
            clearance="+tO",
            no_clip=True,
        )
        fig.text(
            x=stop[0],
            y=stop[1],
            text="A' ",
            fill="white",
            font="12p,Helvetica,black",
            justify="CM",
            clearance="+tO",
            no_clip=True,
        )
    else:
        fig.grdcontour(
            grid=g,
            interval=200,
            annotation=None,
            # pen=".2p,white",
            transparency=20,
            # resample=10e3,
            # cut="1C",
        )

    fig.plot(
        x=constraints[constraints.inside].easting,
        y=constraints[constraints.inside].northing,
        style="x.2c",
        pen="1p,black",
    )
    fig.plot(
        "../../data/Ross_Sea_outline.shp",
        pen="0.8p,black",
    )
    fig.text(
        position="TL",
        text=f"{string.ascii_lowercase[i]}",
        fill="white",
        pen=True,
        font="16p,Helvetica,black",
        offset="j.2/-.6",
        clearance="+tO",
        no_clip=True,
    )

    if i == 1:
        info = polar_utils.set_proj(
            region,
            fig_height=10,
        )
        maps.add_scalebar(fig, region, info[1], position="n0/-.04")
        # maps.add_north_arrow(fig, region, info[1], position="n.45/.07")
fig.show()

In [ ]:
# save final bed
output = final_topo.to_dataset(name="z")
output.to_zarr(
    "../../synthetic_data/Ross_Sea_inverted_bed_survey_noise.zarr",
    encoding={"z": {"compressor": zarr.Blosc()}},
    mode="w",
)